In [1]:
from multicamera_acquisition.acquisition import refactor_acquire_video
from multicamera_acquisition.interfaces.config import (
    partial_config_from_camera_list,
    create_full_camera_default_config,
)


In [ ]:
camera_list = [
    {"name": "side3",  "brand": "basler", "id": "40332773", "gain": 6, "exposure": 1000},
    {"name": "side4",  "brand": "basler", "id": "40347935", "gain": 6, "exposure": 1000},
    {"name": "side2",  "brand": "basler", "id": "40347941", "gain": 6, "exposure": 1000},
    {"name": "side1",  "brand": "basler", "id": "40347943", "gain": 6, "exposure": 1000},
    {"name": "top",    "brand": "basler", "id": "40393557", "gain": 6, "exposure": 1000},
    {"name": "bottom", "brand": "basler", "id": "40393560", "gain": 6, "exposure": 1000},
    {"name": "azure_top",    "brand": "azure", "id": "000500221712", "subordinate_delay_off_master_usec": 0},
    {"name": "azure_bottom", "brand": "azure", "id": "000694421712", "subordinate_delay_off_master_usec": 160},    
]

# Set the trigger behavior
for camera in camera_list[:6]:
    camera["trigger_type"] = "microcontroller"

# Parse the "camera list" into a partial config
partial_new_config = partial_config_from_camera_list(camera_list)

# Add writer configs to each camera config
if writer_type == "nvc":
    try:
        import PyNvCodec as nvc
    except ImportError:
        pytest.skip("PyNvCodec not installed, try running with --writer_type ffmpeg")
    writer_config = NVC_Writer.default_writer_config(fps).copy()
elif writer_type == "ffmpeg":
    writer_config = FFMPEG_Writer.default_writer_config(fps).copy()

for camera_name in partial_new_config["cameras"].keys():
    writer_config["camera_name"] = camera_name
    partial_new_config["cameras"][camera_name]["writer"] = writer_config

# Create the full config, filling in defaults where necessary
full_config = create_full_camera_default_config(partial_new_config, fps)
full_config["globals"] = {}
full_config["globals"]["fps"] = fps
full_config["globals"]["microcontroller_required"] = (
    trigger_type == "microcontroller"
)

# Set up the acquisition loop part of the config
acq_config = AcquisitionLoop.default_acq_loop_config().copy()
acq_config["max_frames_to_acqure"] = n_test_frames
full_config["acq_loop"] = acq_config

# Run the func!
save_loc, first_video_file_name, full_config = refactor_acquire_video(
    tmp_path,
    full_config,
    recording_duration_s=int(n_test_frames / fps),
    append_datetime=True,
    overwrite=False,
    logging_level=logging_level
)
